In [2]:
# Install PyMuPDF
%pip install PyMuPDF

import fitz  # PyMuPDF for extracting text from PDFs
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML  # For displaying images
import pickle

In [3]:

products = pd.DataFrame([
    {"Name": "Ashwagandha Capsules - Stress & Heart Health", "Description": "Ashwagandha helps reduce stress, regulate heart rate, and improve sleep quality.", "Price": 459, "Image URL": "https://m.media-amazon.com/images/I/71GcBHK3utL.jpg"},
    {"Name": "Brahmi Tablets - Mental Wellness & Stress Relief", "Description": "Brahmi enhances mental clarity, reduces anxiety, and stabilizes stress hormones.", "Price": 489, "Image URL": "https://m.media-amazon.com/images/I/51C0+wVHRFL.jpg"},
    {"Name": "Arjuna Powder - Heart Health & Blood Circulation", "Description": "Arjuna is a natural heart tonic that supports cardiovascular health and normalizes BPM.", "Price": 429, "Image URL": "https://m.media-amazon.com/images/I/71vYfp2dKQL.jpg"},
    {"Name": "Giloy Juice - Immunity & Temperature Regulation", "Description": "Giloy strengthens immunity and helps regulate body temperature during fevers.", "Price": 479, "Image URL": "https://m.media-amazon.com/images/I/71mqXwX6yUL.jpg"},
    {"Name": "Shankhpushpi Syrup - Anxiety & Stress Relief", "Description": "Shankhpushpi calms the nervous system and helps relieve anxiety and stress.", "Price": 449, "Image URL": "https://m.media-amazon.com/images/I/81jqfa8oxLL.jpg"},
    {"Name": "Mulethi Churna - Respiratory & Fever Relief", "Description": "Mulethi aids in respiratory health and helps in fever management.", "Price": 439, "Image URL": "https://www.patanjaliayurved.net/assets/product_images/400x500/1721990104mulethichuran1.png"},
    {"Name": "Yashtimadhu Capsules", "Description": "Soothes the nervous system and reduces stress", "Price": 469, "Image URL": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS6uds7WMah0lTK_6b0yCzvWNrWNEj3D2n8tw&s"},
    {"Name": "Punarnava Powder", "Description": "Supports healthy circulation and body temperature", "Price": 419, "Image URL": "https://m.media-amazon.com/images/I/61V2xf8ofFL._AC_UF1000,1000_QL80_.jpg"},
    {"Name": "Amalaki Powder", "Description": "helps reduce stress and balance BPM", "Price": 499, "Image URL": "https://m.media-amazon.com/images/I/81lsvLqDhbL.jpg"},
    {"Name": "Sarpagandha Tablets", "Description": "Helps regulate high blood pressure and BPM", "Price": 489, "Image URL": "https://rukminim2.flixcart.com/image/850/1000/xif0q/shopsy-ayurvedic/9/a/d/sarpagandha-tablet-50-tablets-pack-of-2-1-baidyanath-original-imah5y3hrswevkau.jpeg?q=20&crop=false"},
    {"Name": "Vacha Powder", "Description": "Improves mental clarity and reduces stress", "Price": 479, "Image URL": "https://m.media-amazon.com/images/I/71atWOzL4FL._AC_UF1000,1000_QL80_.jpg"},
    {"Name": "Haritaki Powder", "Description": "Detoxifies the body and balances body temperature", "Price": 469, "Image URL": "https://m.media-amazon.com/images/I/71MGExHk+xL.jpg"},
    {"Name": "Manjistha Powder", "Description": "Supports healthy circulation and reduces stress", "Price": 459, "Image URL": "https://rukminim2.flixcart.com/image/850/1000/l37mtu80/face-pack/8/b/b/150-manjistha-powder-rubia-cordifolia-for-face-skin-hair-care-original-imagedsqfuasewzf.jpeg?q=90&crop=false"},
    {"Name": "Guduchi Powder", "Description": "Boosts immunity and regulates body temperature", "Price": 449, "Image URL": "https://m.media-amazon.com/images/I/71lmNL-YGJL.jpg"},
    {"Name": "Licorice Root Powder", "Description": "Soothes the nervous system and reduces stress", "Price": 439, "Image URL": "https://m.media-amazon.com/images/I/61-t1g9tNRL._AC_UF1000,1000_QL80_.jpg"},
    {"Name": "Shatavari Powder", "Description": "Helps balance hormones and reduce stress", "Price": 429, "Image URL": "https://m.media-amazon.com/images/I/711qGODnWxL.jpg"},
    {"Name": "Moringa Powder", "Description": "supports heart health and reduces stress", "Price": 419, "Image URL": "https://m.media-amazon.com/images/I/71kaWiBZkNL.jpg"},
    {"Name": "Triphala Powder", "Description": "Detoxifies the body and supports overall health", "Price": 499, "Image URL": "https://m.media-amazon.com/images/I/81qDvBBnDXL.jpg"},
    {"Name": "Dashmool Powder", "Description": "Reduces stress and supports joint health", "Price": 489, "Image URL": "https://m.media-amazon.com/images/I/81Q9XJmvgDL.jpg"},
    {"Name": "Chyawanprash", "Description": "Boosts immunity and regulates body temperature", "Price": 479, "Image URL": "https://m.media-amazon.com/images/I/71tH6y4K-SL._AC_UF1000,1000_QL80_.jpg"},
    {"Name": "Bala Oil", "Description": "Promotes relaxation and supports heart health", "Price": 469, "Image URL": "https://m.media-amazon.com/images/I/51+-z4cm3DL.jpg"},
    {"Name": "Bhringraj Oil", "Description": "Reduces stress and supports hair health", "Price": 459, "Image URL": "https://www.greenlifeherbals.co/cdn/shop/products/004.jpg?v=1681288264"},
    {"Name": "Neem Capsules", "Description": "Detoxifies the body and regulates body temperature", "Price": 449, "Image URL": "https://m.media-amazon.com/images/I/71frSTC2ybL.jpg"},
    {"Name": "Turmeric Powder", "Description": "Reduces inflammation and supports heart health", "Price": 439, "Image URL": "https://www.tatanutrikorner.com/cdn/shop/files/turmeric_200g.webp?v=1731900341&width=1445"},
    {"Name": "Ginger Powder", "Description": "Regulates body temperature and supports digestion", "Price": 429, "Image URL": "https://m.media-amazon.com/images/I/81nKpe0VBPL.jpg"},
    {"Name": "Cinnamon Powder", "Description": "Supports healthy circulation and reduces stress", "Price": 419, "Image URL": "https://rukminim2.flixcart.com/image/850/1000/kz1lle80/spice-masala/h/0/r/200-cinnamon-powder-dalchini-powder-1-pouch-nutri-guide-powder-original-imagb548guqjz7h4.jpeg?q=20&crop=false"},
    {"Name": "Cardamom Powder", "Description": "Calms the mind and supports heart health", "Price": 499, "Image URL": "https://keyafoods.com/cdn/shop/files/CardemonSeedPowder.jpg?v=1719815049&width=1445"},
    {"Name": "Fennel Powder", "Description": "Supports digestion and reduces stress", "Price": 489, "Image URL": "https://aachifoods.com/cdn/shop/files/Fennel-Powder.webp?v=1726910940"},
    {"Name": "Cumin Powder", "Description": "Regulates body temperature and supports digestion", "Price": 479, "Image URL": "https://m.media-amazon.com/images/I/81Y4dOt-V1L.jpg"},
    {"Name": "Ajwain Powder", "Description": "Supports digestion and reduces stress", "Price": 469, "Image URL": "https://vedcare.in/wp-content/uploads/2023/12/Vedcare-Natural-Ajwain-Carom-Seed-Powder-200-gm.jpg"},
    {"Name": "Peppermint Oil", "Description": "Calms the mind and regulates body temperature", "Price": 459, "Image URL": "https://m.media-amazon.com/images/I/61AS9xbq0HL._AC_UF1000,1000_QL80_.jpg"},
    {"Name": "Eucalyptus Oil", "Description": "Supports respiratory health and reduces stress", "Price": 449, "Image URL": "https://m.media-amazon.com/images/I/51eEhuIEJ7L.jpg"},
    {"Name": "Lavender Oil", "Description": "Promotes relaxation and reduces stress", "Price": 439, "Image URL": "https://m.media-amazon.com/images/I/81dWT3VBxYL._AC_UF1000,1000_QL80_.jpg"},
    {"Name": "Sandalwood Oil", "Description": "Calms the mind and supports heart health", "Price": 429, "Image URL": "https://m.media-amazon.com/images/I/71yvVbJpAkL.jpg"},
    {"Name": "Rose Water", "Description": "Soothes the skin and reduces stress", "Price": 419, "Image URL": "https://m.media-amazon.com/images/I/81qtIkTFGzL._AC_UF1000,1000_QL80_.jpg"},
    {"Name": "Aloe Vera Gel", "Description": "Regulates body temperature and soothes the skin", "Price": 499, "Image URL": "https://m.media-amazon.com/images/I/61A58BcmbmL.jpg"},
    {"Name": "Honey", "Description": "Boosts immunity and supports heart health", "Price": 489, "Image URL": "https://www.bigbasket.com/media/uploads/p/xxl/239524-2_6-dabur-100-pure-honey-worlds-no1-honey-brand-with-no-sugar-adulteration.jpg"},
    {"Name": "Ghee", "Description": "Supports digestion and reduces stress", "Price": 479, "Image URL": "https://www.shuddhamorganics.com/cdn/shop/files/SmartSelect-20231001_000630_Gallery.jpg?v=1696099042"},
    {"Name": "Sesame Oil", "Description": "Promotes relaxation and supports heart health", "Price": 469, "Image URL": "https://organicmandya.com/cdn/shop/files/200ml_Sesame_Oil.jpg?v=1724933834&width=1500"},
    {"Name": "Coconut Oil", "Description": "Regulates body temperature and supports skin health", "Price": 459, "Image URL": "https://m.media-amazon.com/images/I/71+I2IPZOqL.jpg"},
    {"Name": "Mustard Oil", "Description": "Supports heart health and reduces stress", "Price": 449, "Image URL": "https://m.media-amazon.com/images/I/71uXfQr+SjL._AC_UF350,350_QL80_.jpg"},
    {"Name": "Castor Oil", "Description": "Detoxifies the body and supports digestion", "Price": 439, "Image URL": "https://m.media-amazon.com/images/I/71Pem0dt08L.jpg"},
    {"Name": "Amla Juice", "Description": "supports heart health and reduces stress", "Price": 429, "Image URL": "https://m.media-amazon.com/images/I/71cqC6167lL.jpg"},
    {"Name": "Pomegranate Juice", "Description": "Supports heart health and reduces stress", "Price": 419, "Image URL": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTKbo0mq3PS1WLybkIeMPm0gIbVUrr8xvm1vg&s"},
    {"Name": "Tamarind Powder", "Description": "Supports digestion and reduces stress", "Price": 489, "Image URL": "https://m.media-amazon.com/images/I/71ZNbfUQxjL.jpg"},
    {"Name": "Basil Oil", "Description": "Calms the mind and supports heart health", "Price": 479, "Image URL": "https://cloudinary.images-iherb.com/image/upload/f_auto,q_auto:eco/images/lat/lat00561/y/34.jpg"},
    {"Name": "Clove Oil", "Description": "Supports respiratory health and reduces stress", "Price": 469, "Image URL": "https://m.media-amazon.com/images/I/81l+C7zTvwL.jpg"}
])


In [4]:

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

In [5]:
def preprocess_health_report(text):
    """Extract key health parameters from the health report text."""
    health_params = {}

    # Extract heart rate
    if "Heart Rate:" in text:
        lines = text.split("\n")
        for line in lines:
            if "Current:" in line:
                health_params["Heart Rate"] = int(line.split(":")[-1].strip().split()[0])
            if "Stress Level:" in line:
                health_params["Stress Level"] = int(line.split(":")[-1].strip())
            if "Body Temperature:" in line:
                health_params["Body Temperature"] = float(line.split(":")[-1].strip().replace("°F", ""))

    return health_params

In [6]:
def recommend_ayurvedic_products(health_params):
    """Recommend top 6 Ayurvedic products based on health conditions."""
    
    # Create user query from health parameters
    query = []
    if health_params.get("Heart Rate"):
        query.append(f"regulates heart rate normalizes BPM")
    if health_params.get("Stress Level") and health_params["Stress Level"] > 50:
        query.append(f"reduces stress relieves anxiety calms nervous system")
    if health_params.get("Body Temperature") and (health_params["Body Temperature"] < 97 or health_params["Body Temperature"] > 99):
        query.append(f"regulates body temperature boosts immunity")

    query_text = " ".join(query)
    descriptions = products["Description"].tolist()
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(descriptions)
    query_vector = vectorizer.transform([query_text])

    # Compute cosine similarity
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Get top 6 recommendations
    top_indices = np.argsort(similarity_scores)[-6:][::-1]
    recommendations = products.iloc[top_indices]

    return recommendations

In [7]:
def display_recommendations(recommended_products):
    """Display product recommendations with images."""
    html_output = "<h2>Top 6 Recommended Ayurvedic Products</h2><table>"
    for _, row in recommended_products.iterrows():
        html_output += f"""
        <tr>
            <td><img src="{row['Image URL']}" width="100" height="100"></td>
            <td><b>{row['Name']}</b><br>{row['Description']}<br><b>Price:</b> ₹{row['Price']}</td>
        </tr>
        """
    html_output += "</table>"
    display(HTML(html_output))

In [8]:
pdf_path = r"C:\Users\singh\Downloads\health_report (6).pdf"  # Replace with uploaded PDF path
report_text = extract_text_from_pdf(pdf_path)
health_data = preprocess_health_report(report_text)
recommended_products = recommend_ayurvedic_products(health_data)

In [9]:
display_recommendations(recommended_products)

,Shankhpushpi Syrup - Anxiety & Stress ReliefShankhpushpi calms the nervous system and helps relieve anxiety and stress.Price: ₹449
,Licorice Root PowderSoothes the nervous system and reduces stressPrice: ₹439
,Yashtimadhu CapsulesSoothes the nervous system and reduces stressPrice: ₹469
,Arjuna Powder - Heart Health & Blood CirculationArjuna is a natural heart tonic that supports cardiovascular health and normalizes BPM.Price: ₹429
,Mustard OilSupports heart health and reduces stressPrice: ₹449
,Amla Juicesupports heart health and reduces stressPrice: ₹429


In [10]:
pickle.dump(recommended_products, open("recommended_products.pkl", "wb"))

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_recommendations(health_params, recommended_products):
    """Evaluate recommendation system's performance."""
    
    # Define ground truth (actual expected categories based on health conditions)
    ground_truth = []
    if health_params.get("Heart Rate"):
        ground_truth.append("heart rate")
    if health_params.get("Stress Level") and health_params["Stress Level"] > 50:
        ground_truth.append("stress")
    if health_params.get("Body Temperature") and (health_params["Body Temperature"] < 97 or health_params["Body Temperature"] > 99):
        ground_truth.append("temperature")

    # Extract predicted categories from recommended products
    predicted_categories = []
    for _, row in recommended_products.iterrows():
        if "heart" in row["Description"].lower():
            predicted_categories.append("heart rate")
        if "stress" in row["Description"].lower() or "anxiety" in row["Description"].lower():
            predicted_categories.append("stress")
        if "temperature" in row["Description"].lower() or "immunity" in row["Description"].lower():
            predicted_categories.append("temperature")

    # Convert to binary arrays for precision/recall calculation
    all_categories = list(set(ground_truth + predicted_categories))
    y_true = [1 if category in ground_truth else 0 for category in all_categories]
    y_pred = [1 if category in predicted_categories else 0 for category in all_categories]

    # Calculate evaluation metrics
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"🔍 Precision: {precision:.2f}")
    print(f"📡 Recall: {recall:.2f}")
    print(f"🎯 F1-Score: {f1:.2f}")

    return precision, recall, f1

# Evaluate recommendations
precision, recall, f1 = evaluate_recommendations(health_data, recommended_products)


🔍 Precision: 1.00
📡 Recall: 1.00
🎯 F1-Score: 1.00
